In [1]:
import pandas as pd
#import files into DataFrames using pandas
cabData = pd.read_csv('Cab_Data.csv')
cityData = pd.read_csv('City.csv')
customerData = pd.read_csv('Customer_ID.csv')
transactionData = pd.read_csv('Transaction_ID.csv')
transactionData = transactionData.merge(customerData, on="Customer ID", how='left')
cabData = cabData.merge(transactionData, on="Transaction ID", how="left")
cabData = cabData.merge(cityData, on="City", how="left")
print(cabData)

        Transaction ID  Date of Travel     Company           City  \
0             10000011           42377    Pink Cab     ATLANTA GA   
1             10000012           42375    Pink Cab     ATLANTA GA   
2             10000013           42371    Pink Cab     ATLANTA GA   
3             10000014           42376    Pink Cab     ATLANTA GA   
4             10000015           42372    Pink Cab     ATLANTA GA   
...                ...             ...         ...            ...   
359387        10440101           43108  Yellow Cab  WASHINGTON DC   
359388        10440104           43104  Yellow Cab  WASHINGTON DC   
359389        10440105           43105  Yellow Cab  WASHINGTON DC   
359390        10440106           43105  Yellow Cab  WASHINGTON DC   
359391        10440107           43102  Yellow Cab  WASHINGTON DC   

        KM Travelled  Price Charged  Cost of Trip  Customer ID Payment_Mode  \
0              30.45         370.95      313.6350        29290         Card   
1            